In [140]:
import pandas as pd
import pyexasol as pye
import pandas_profiling
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import missingno as msno
import plotly.graph_objects as go
import plotly

In [95]:
C = pye.connect(dsn='localhost:8564', user='mantasrazanauskas', password='5DZVYQMEUNGT5Y9JWD9F', compression=True)

In [103]:
#get data from DB with all accounts with 'team' tag
team_tag = C.export_to_pandas("""Select *
from raw.user_tags as ut 
join raw.users u on u.id = ut.id
where tag = 'team'""")

In [14]:
employees = pd.read_csv('/Users/mantas.razanauskas/Audits/2020/Fraud/employee_details.csv')

In [15]:
employees

,user_full_name,mobile_number,private_phone,private_email
0,Michael Thomas,+447867484593,NaN,michaelthomas@sky.com
1,Juan Antonio Gomez Garza,+447494645854,+1 415 812 5261,antonio.gomezgarza@gmail.com
2,Shifnaz Ifham,NaN,NaN,shifnaz.ifham@revolut.com
3,Aleksei Savin,+447564655631,+447564655631,leshasavin@gmail.com
4,Melani Georgieva,07706780779,NaN,melanie.georgieva@gmail.com
...,...,...,...,...
2170,Natalia Grudina,+44 7379 406008,NaN,grudina.natalia@gmail.com
2171,Eleni Filippakopoulou,+447426096949,NaN,elefilipo@gmail.com
2172,Rifat Ara Shaikh,07889445675,NaN,rifatshaikh@hotmail.co.uk
2173,Katarzyna Cierniak,+447464254486,+447464254486,kj.cierniak@gmail.com


In [188]:
source_data = {'Node, Label':['team tagged in BO','Employees','Tagged non-employees','Not tagged employees'], 
             'Color':['#db8c2c','#db8c2c' ,'#b51212','#fc8686'], #base color in hex
             'Source':[0,1,0,1],
             'Target':[2,2,3,4],
             'Value':[2705, 2175, 814, 284],
              'Color2':['#82baff','#3d8ff5','#e89090','#fc8686'] # line color in hex
            }

source_df = pd.DataFrame(source_data)
source_df

,"Node, Label",Color,Source,Target,Value,Color2
0,team tagged in BO,#db8c2c,0,2,2705,#82baff
1,Employees,#db8c2c,1,2,2175,#3d8ff5
2,Tagged non-employees,#b51212,0,3,814,#e89090
3,Not tagged employees,#fc8686,1,4,284,#fc8686


In [189]:

fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    # Define nodes
    node = dict(
      pad = 20,
      thickness = 15,
      line = dict(color = "black", width = 0.7),
      label = ['team tagged in BO (2705)','Employees (2175)','Tagged employees (1891)','Tagged non-employees (814)','Not tagged employees (284)'], #update this to update labels
      color =  ['#2a5431','#2a5431','#b51212','#b51212','#237826']
    ),
    # Add links
    link = dict(
      source =  source_df['Source'],
      target =  source_df['Target'],
      value =  source_df['Value'],
      label =  source_df['Node, Label'],
      color =  source_df['Color2']
))])

fig.update_layout(title_text=""""team" tag <br>Analysis""",
                  font_size=15)
fig.show()

In [81]:
non_employees = pd.read_csv('/Users/mantas.razanauskas/Audits/2020/Fraud/non_employees_tag.csv')

In [85]:
list_nonemployees = list(non_employees['ID'])
query = """Select id, created_date, plan, country, city, email  from raw.users where id in {} """.format(tuple(list_nonemployees))


In [138]:
plan = C.export_to_pandas(query)



In [151]:
plan_counts = plan.groupby(['PLAN']).count()
plan_counts

,ID,CREATED_DATE,COUNTRY,CITY,EMAIL
PLAN,,,,,
FREELANCE_FREE,32,32,32,32,0
FREELANCE_PRO,25,25,25,25,0
FREELANCE_STARTER,39,39,39,39,0
METAL,529,529,529,529,529
METAL_FREE,11,11,11,11,11
METAL_FREE_OLD,1,1,1,1,1
PREMIUM,40,40,40,40,40
PREMIUM_FREE,9,9,9,9,9
PREMIUM_FREE_OLD,1,1,1,1,1


In [154]:
list(plan_counts.index)

['FREELANCE_FREE',
 'FREELANCE_PRO',
 'FREELANCE_STARTER',
 'METAL',
 'METAL_FREE',
 'METAL_FREE_OLD',
 'PREMIUM',
 'PREMIUM_FREE',
 'PREMIUM_FREE_OLD',
 'STANDARD']

In [158]:
fig = px.bar(plan_counts, x=list(plan_counts.index), y="ID",title="Non-employees Plan")
fig.show()

In [165]:
month = [1,2,3,4,5,6,7,8,9,10,11,12]

In [166]:
metal_counts = plan_counts['ID'][3]
premium_counts = plan_counts['ID'][6]

In [176]:
my_df = {'Month':month, 'Metal':metal_counts, 'Premium':premium_counts, 'Metal_price':12.99, 'Premium_price':6.99}
revenue = pd.DataFrame(my_df)

In [180]:
revenue['Lost_metal_revenue'] = revenue['Month']*revenue['Metal']*revenue['Metal_price']
revenue['Lost_premium_revenue'] = revenue['Month']*revenue['Premium']*revenue['Premium_price']
revenue['Total_revenue_lost'] = revenue['Lost_metal_revenue'] + revenue['Lost_premium_revenue']

In [181]:
revenue

,Month,Metal,Premium,Metal_price,Premium_price,Lost_metal_revenue,Lost_premium_revenue,Total_revenue_lost
0,1,529,40,12.99,6.99,6871.71,279.6,7151.31
1,2,529,40,12.99,6.99,13743.42,559.2,14302.62
2,3,529,40,12.99,6.99,20615.13,838.8,21453.93
3,4,529,40,12.99,6.99,27486.84,1118.4,28605.24
4,5,529,40,12.99,6.99,34358.55,1398.0,35756.55
5,6,529,40,12.99,6.99,41230.26,1677.6,42907.86
6,7,529,40,12.99,6.99,48101.97,1957.2,50059.17
7,8,529,40,12.99,6.99,54973.68,2236.8,57210.48
8,9,529,40,12.99,6.99,61845.39,2516.4,64361.79
9,10,529,40,12.99,6.99,68717.10,2796.0,71513.10


In [185]:

fig = px.line(revenue, x="Month", y=["Total_revenue_lost",'Lost_metal_revenue','Lost_premium_revenue'], title = 'Potential revenue loss')
fig.show()